In [ ]:
def ga_scraper(login, password, accountName, downloadPath,
              startDate, endDate, lang):
    return visitorsFilesList

In [33]:
# importing libraries
import selenium
import time
import os
import os.path
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd 

In [36]:
# setting login, password etc.
login =''
password = ''
audience = 'АУДИТОРИЯ'
userStastics = 'Статистика по пользователям'
startDate = '01.03.2017'
endDate = '31.03.2017'
downloadPath = 'C:/Users/kudinov/Documents/Google_Analytics/Sandbox/jsons_03_2017'

In [ ]:
# changing default download directory

chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : downloadPath}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path=r"chromedriver.exe", chrome_options=chromeOptions)

# going to the home web page
#driver = webdriver.Chrome(executable_path=r"chromedriver.exe")
driver.get("https://analytics.google.com/analytics/web/")



# log-in (passing login) and clicking next (to the password) 
driver.find_element_by_id("identifierId").clear()
driver.find_element_by_id("identifierId").send_keys(login)
driver.find_element_by_id("identifierNext").click()


# passing password 
## Unworkable code -- driver.find_element_by_id("passwrd").send_keys(password)
## Unworkable code -- driver.find_element_by_id("passwordNext").click()driver.find_element_by_name("password").send_keys(password)
try:
    element = WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.NAME, 'password'))
    )
finally:
    #driver.quit()
    print('Sobyakin asshole. Once')
    
driver.find_element_by_name("password").send_keys(password)
element = driver.find_element_by_id('passwordNext')
driver.execute_script("arguments[0].click();", element)

# navigating to the target web page, step 1
try:
    element = WebDriverWait(driver, 60).until(
        EC.text_to_be_present_in_element((By.PARTIAL_LINK_TEXT, audience), audience)
    )
finally:
    #driver.quit()
    print('Sobyakin asshole. Twice')
driver.find_element_by_partial_link_text(audience).click()

# navigating to target web page, step 2
try:
    element = WebDriverWait(driver, 60).until(
        EC.text_to_be_present_in_element((By.PARTIAL_LINK_TEXT, userStastics), userStastics)
    )
finally:
    #driver.quit()
    print('Sobyakin asshole. Thrise')
driver.find_element_by_partial_link_text(userStastics).click()

# switching to the iframe by its name (galaxy) - that contains the right part of the page
try:
    element = WebDriverWait(driver, 60).until(
        EC.visibility_of_element_located((By.NAME, 'galaxy'))
    )
finally:
    #driver.quit()
    print('Sobyakin asshole. Forth')
driver.switch_to.frame(driver.find_element_by_name('galaxy'))

# activating datepicker by finding it by classs name and clicking with executing script
try:
    element = WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.CLASS_NAME, '_GArn'))
    )
finally:
    #driver.quit()
    print('Sobyakin asshole. Five')

driver.find_element_by_class_name('_GArn').click()

# selecting start and end dates
driver.find_element_by_class_name('ID-datecontrol-primary-start').clear()
driver.find_element_by_class_name('ID-datecontrol-primary-start').send_keys(startDate)
driver.find_element_by_class_name('ID-datecontrol-primary-end').clear()
driver.find_element_by_class_name('ID-datecontrol-primary-end').send_keys(endDate)
driver.find_element_by_class_name('ID-apply').click()

# displaying all the rows on the page
try:
    element = WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.CLASS_NAME, 'ACTION-toggleRowShow'))
    )
finally:
    #driver.quit()
    print('Sobyakin asshole. Six')
driver.find_element_by_class_name("ACTION-toggleRowShow").send_keys(5000)

# downloading the top level summary table "Статистика по пользователям"
time.sleep(5)
driver.find_element_by_class_name("ID-exportControlButton").click()
li = driver.find_elements_by_class_name("ACTION-export")
li[2].click()

# getting number of visitors' IDs from the summary table
elem = driver.find_elements_by_class_name("C_USER_LIST_TEXT_DIV")
static_len = len(elem)

summaryTableURL = driver.current_url

# renaming the downloaded file with the summary table to ClientSummary.xlsx
clientSummaryFileName = '{2}{3}{0}{3}{1}{4}'.format(startDate, endDate, 'clientSummary', '_', '.xlsx')
os.rename('{0}{2}{1}'.format(downloadPath, os.listdir(downloadPath)[0],'/'),
          '{0}{2}{1}'.format(downloadPath, clientSummaryFileName,'/'))


# loading the 1st column (IDs) from the summary table (ClientSummary.xlsx) dataFrame clientsID
clientsID = pd.read_excel(io = '{0}{2}{1}'.format(downloadPath, clientSummaryFileName,'/'), sheet_name = 'Набор данных1',
                             usecols = 0, dtype = {'Идентификатор клиента' : str})

for number in range(0,static_len):
    try:
        element = WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.CLASS_NAME, 'C_USER_LIST_TEXT_DIV'))    )
    finally:
            #driver.quit()
        print('Sobyakin asshole. seven')
    #time.sleep(5)
    elem = driver.find_elements_by_class_name("C_USER_LIST_TEXT_DIV")
    elem[number].click()
#    time.sleep(5)
    try:
        item = WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.CLASS_NAME, '_GAGW'))    )
    finally:
            #driver.quit()
        print('Sobyakin asshole. eight')
    item = driver.find_elements_by_class_name("_GAGW")
    item[5].click()
    
    while not os.path.exists('{0}{1}{2}'.format(downloadPath, '/','user-report-export.json')):
        time.sleep(3)
    if os.path.isfile('{0}{1}{2}'.format(downloadPath, '/','user-report-export.json')):
        os.rename('{0}{1}{2}'.format(downloadPath, '/','user-report-export.json'),
                  '{0}{2}{1}{3}'.format(downloadPath, clientsID.iat[number,0],'/','.json'))
    else:
        raise ValueError("%s isn't a file!" % '{0}{1}{2}'.format(downloadPath, '/','user-report-export.json'))    
    
    
    driver.get(summaryTableURL)



#    driver.back()
#    driver.back()
#    driver.back()
#    driver.back()
#    driver.back()
    try:
        element = WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.NAME, 'galaxy'))    )
    finally:
            #driver.quit()
        print(number)
#    time.sleep(15)
    driver.switch_to.frame(driver.find_element_by_name('galaxy'))
    
driver.close()

Sobyakin asshole. Once
Sobyakin asshole. Twice
Sobyakin asshole. Thrise
Sobyakin asshole. Forth
Sobyakin asshole. Five
Sobyakin asshole. Six
Sobyakin asshole. seven
Sobyakin asshole. eight
0
Sobyakin asshole. seven
Sobyakin asshole. eight
1
Sobyakin asshole. seven
Sobyakin asshole. eight
2
Sobyakin asshole. seven
Sobyakin asshole. eight
3
Sobyakin asshole. seven
Sobyakin asshole. eight
4
Sobyakin asshole. seven
Sobyakin asshole. eight
5
Sobyakin asshole. seven
Sobyakin asshole. eight
6
Sobyakin asshole. seven
Sobyakin asshole. eight
7
Sobyakin asshole. seven
Sobyakin asshole. eight
8
Sobyakin asshole. seven
Sobyakin asshole. eight
9
Sobyakin asshole. seven
Sobyakin asshole. eight
10
Sobyakin asshole. seven
Sobyakin asshole. eight
11
Sobyakin asshole. seven
Sobyakin asshole. eight
12
Sobyakin asshole. seven
Sobyakin asshole. eight
13
Sobyakin asshole. seven
Sobyakin asshole. eight
14
Sobyakin asshole. seven
Sobyakin asshole. eight
15
Sobyakin asshole. seven
Sobyakin asshole. eight
16
So

Sobyakin asshole. seven
Sobyakin asshole. eight
157
Sobyakin asshole. seven
Sobyakin asshole. eight
158
Sobyakin asshole. seven
Sobyakin asshole. eight
159
Sobyakin asshole. seven
Sobyakin asshole. eight
160
Sobyakin asshole. seven
Sobyakin asshole. eight
161
Sobyakin asshole. seven
Sobyakin asshole. eight
162
Sobyakin asshole. seven
Sobyakin asshole. eight
163
Sobyakin asshole. seven
Sobyakin asshole. eight
164
Sobyakin asshole. seven
Sobyakin asshole. eight
165
Sobyakin asshole. seven
Sobyakin asshole. eight
166
Sobyakin asshole. seven
Sobyakin asshole. eight
167
Sobyakin asshole. seven
Sobyakin asshole. eight
168
Sobyakin asshole. seven
Sobyakin asshole. eight
169
Sobyakin asshole. seven
Sobyakin asshole. eight
170
Sobyakin asshole. seven
Sobyakin asshole. eight
171
Sobyakin asshole. seven
Sobyakin asshole. eight
172
Sobyakin asshole. seven
Sobyakin asshole. eight
173
Sobyakin asshole. seven
Sobyakin asshole. eight
174
Sobyakin asshole. seven
Sobyakin asshole. eight
175
Sobyakin ass

314
Sobyakin asshole. seven
Sobyakin asshole. eight
315
Sobyakin asshole. seven
Sobyakin asshole. eight
316
Sobyakin asshole. seven
Sobyakin asshole. eight
317
Sobyakin asshole. seven
Sobyakin asshole. eight
318
Sobyakin asshole. seven
Sobyakin asshole. eight
319
Sobyakin asshole. seven
Sobyakin asshole. eight
320
Sobyakin asshole. seven
Sobyakin asshole. eight
321
Sobyakin asshole. seven
Sobyakin asshole. eight
322
Sobyakin asshole. seven
Sobyakin asshole. eight
323
Sobyakin asshole. seven
Sobyakin asshole. eight
324
Sobyakin asshole. seven
Sobyakin asshole. eight
325
Sobyakin asshole. seven
Sobyakin asshole. eight
326
Sobyakin asshole. seven
Sobyakin asshole. eight
327
Sobyakin asshole. seven
Sobyakin asshole. eight
328
Sobyakin asshole. seven
Sobyakin asshole. eight
329
Sobyakin asshole. seven
Sobyakin asshole. eight
330
Sobyakin asshole. seven
Sobyakin asshole. eight
331
Sobyakin asshole. seven
Sobyakin asshole. eight
332
Sobyakin asshole. seven
Sobyakin asshole. eight
333
Sobyakin

472
Sobyakin asshole. seven
Sobyakin asshole. eight
473
Sobyakin asshole. seven
Sobyakin asshole. eight
474
Sobyakin asshole. seven
Sobyakin asshole. eight
475
Sobyakin asshole. seven
Sobyakin asshole. eight
476
Sobyakin asshole. seven
Sobyakin asshole. eight
477
Sobyakin asshole. seven
Sobyakin asshole. eight
478
Sobyakin asshole. seven
Sobyakin asshole. eight
479
Sobyakin asshole. seven
Sobyakin asshole. eight
480
Sobyakin asshole. seven
Sobyakin asshole. eight
481
Sobyakin asshole. seven
Sobyakin asshole. eight
482
Sobyakin asshole. seven
Sobyakin asshole. eight
483
Sobyakin asshole. seven
Sobyakin asshole. eight
484
Sobyakin asshole. seven
Sobyakin asshole. eight
485
Sobyakin asshole. seven
Sobyakin asshole. eight
486
Sobyakin asshole. seven
Sobyakin asshole. eight
487
Sobyakin asshole. seven
Sobyakin asshole. eight
488
Sobyakin asshole. seven
Sobyakin asshole. eight
489
Sobyakin asshole. seven
Sobyakin asshole. eight
490
Sobyakin asshole. seven
Sobyakin asshole. eight
491
Sobyakin

In [38]:
driver.close()

In [25]:
rows_elements = driver.find_element_by_class_name("C_PAGINATION_ROWS_LONG")

In [26]:
rows_elements.get_attribute('innerHTML')

'<span class="_GABG"><label>Строк на странице:</label><select class="ACTION-toggleRowShow TARGET-"><option value="10">10</option><option value="25">25</option><option value="50">50</option><option value="100">100</option><option value="250">250</option><option value="500">500</option><option value="1000">1000</option><option value="2500">2500</option><option value="5000" selected="true">5000</option></select></span><span><label>К строке:</label><input class="ID-gotoInput" type="text" value="1" maxlength="5"></span><label>1–1612 из 1612</label>'